# Day 5: Building Neural Networks from First Principles (NumPy)

**Time:** 4-5 hours (substantial implementation)

**Mathematical Prerequisites:**
- Matrix calculus (Jacobians, vector-Jacobian products)
- Chain rule for compositions
- Computational graphs and reverse-mode autodiff
- Numerical methods and stability

---

## Objectives

Today we implement a complete neural network library from scratch using only NumPy. This will deepen your understanding of:
1. How automatic differentiation actually works
2. Forward and backward propagation at a low level
3. The mathematics behind gradient computation
4. Why reverse-mode autodiff is O(n) not O(n²)

By the end, you'll have a working NN library that achieves >90% accuracy on MNIST.

---

## Part 1: Mathematical Foundation

### 1.1 Computational Graphs and the Chain Rule

A neural network is a composition of functions:
$$
f(x; \theta) = f_L \circ f_{L-1} \circ \cdots \circ f_1(x)
$$

For a loss $L(f(x; \theta), y)$, we need $\frac{\partial L}{\partial \theta}$ for gradient descent.

**Chain Rule (Vector Calculus):**

If $y = f(x)$ where $y \in \mathbb{R}^m$, $x \in \mathbb{R}^n$, then:
$$
\frac{\partial L}{\partial x} = \left(\frac{\partial y}{\partial x}\right)^T \frac{\partial L}{\partial y}
$$

where $\frac{\partial y}{\partial x}$ is the Jacobian (m×n matrix).

**Key Insight:** In reverse-mode autodiff, we propagate **vector-Jacobian products** (VJPs) backward. This is why backpropagation is O(n) not O(n²) - we never explicitly compute the full Jacobian.

### 1.2 Forward and Backward Pass

**Forward pass:** Compute outputs and cache intermediate values
```python
y = f(x)  # Store x for backward pass
```

**Backward pass:** Given $\frac{\partial L}{\partial y}$ (gradient from next layer), compute:
1. $\frac{\partial L}{\partial x}$ to pass to previous layer
2. $\frac{\partial L}{\partial \theta}$ to update parameters

```python
dL/dx = (dy/dx)^T @ (dL/dy)  # VJP
dL/dθ = (dy/dθ)^T @ (dL/dy)  # Parameter gradient
```

### 1.3 Example: Linear Layer

Forward: $y = Wx + b$ where $W \in \mathbb{R}^{m \times n}$, $x \in \mathbb{R}^n$, $b \in \mathbb{R}^m$

Gradients:
$$
\frac{\partial L}{\partial W} = \frac{\partial L}{\partial y} x^T \quad (m \times n)
$$
$$
\frac{\partial L}{\partial b} = \frac{\partial L}{\partial y} \quad (m \times 1)
$$
$$
\frac{\partial L}{\partial x} = W^T \frac{\partial L}{\partial y} \quad (n \times 1)
$$

**For batched inputs** $X \in \mathbb{R}^{B \times n}$:
$$
Y = XW^T + b
$$
$$
\frac{\partial L}{\partial W} = \frac{1}{B} \left(\frac{\partial L}{\partial Y}\right)^T X
$$
$$
\frac{\partial L}{\partial b} = \frac{1}{B} \sum_{i=1}^B \frac{\partial L}{\partial y_i}
$$
$$
\frac{\partial L}{\partial X} = \frac{\partial L}{\partial Y} W
$$

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
import seaborn as sns
import time

# For comparison with PyTorch later
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

np.random.seed(42)

## Part 2: Layer Abstraction

We'll create a base `Layer` class that all layers inherit from. Each layer must implement:
- `forward(x)`: Compute output and cache inputs
- `backward(grad_output)`: Compute gradients w.r.t. inputs and parameters

In [ ]:
class Layer:
    """Base class for all layers."""
    
    def __init__(self):
        self.params = {}  # Parameters (weights, biases)
        self.grads = {}   # Gradients of parameters
        self.cache = {}   # Cache for backward pass
    
    def forward(self, x):
        """Forward pass. Must be implemented by subclasses.
        
        Args:
            x: Input array of shape (batch_size, input_dim)
        
        Returns:
            Output array of shape (batch_size, output_dim)
        """
        raise NotImplementedError
    
    def backward(self, grad_output):
        """Backward pass. Must be implemented by subclasses.
        
        Args:
            grad_output: Gradient w.r.t. output (dL/dy), shape (batch_size, output_dim)
        
        Returns:
            grad_input: Gradient w.r.t. input (dL/dx), shape (batch_size, input_dim)
        """
        raise NotImplementedError
    
    def get_params(self):
        """Return dictionary of parameters."""
        return self.params
    
    def get_grads(self):
        """Return dictionary of gradients."""
        return self.grads

### 2.1 Linear (Fully Connected) Layer

**Forward:** $y = xW^T + b$

**Backward:**
- $\frac{\partial L}{\partial W} = (\frac{\partial L}{\partial y})^T x$
- $\frac{\partial L}{\partial b} = \sum_{batch} \frac{\partial L}{\partial y}$
- $\frac{\partial L}{\partial x} = \frac{\partial L}{\partial y} W$

In [ ]:
class Linear(Layer):
    """Fully connected layer: y = xW^T + b
    
    Args:
        input_dim: Number of input features
        output_dim: Number of output features
    """
    
    def __init__(self, input_dim, output_dim):
        super().__init__()
        
        # Xavier initialization: Var(W) = 2/(fan_in + fan_out)
        limit = np.sqrt(6 / (input_dim + output_dim))
        self.params['W'] = np.random.uniform(-limit, limit, (output_dim, input_dim))
        self.params['b'] = np.zeros(output_dim)
        
        # Initialize gradients
        self.grads['W'] = np.zeros_like(self.params['W'])
        self.grads['b'] = np.zeros_like(self.params['b'])
    
    def forward(self, x):
        """
        Args:
            x: (batch_size, input_dim)
        Returns:
            y: (batch_size, output_dim)
        """
        self.cache['x'] = x
        return x @ self.params['W'].T + self.params['b']
    
    def backward(self, grad_output):
        """
        Args:
            grad_output: dL/dy, shape (batch_size, output_dim)
        Returns:
            grad_input: dL/dx, shape (batch_size, input_dim)
        """
        x = self.cache['x']
        batch_size = x.shape[0]
        
        # Gradient w.r.t. parameters
        self.grads['W'] = (grad_output.T @ x) / batch_size
        self.grads['b'] = np.mean(grad_output, axis=0)
        
        # Gradient w.r.t. input
        grad_input = grad_output @ self.params['W']
        
        return grad_input

### 2.2 ReLU Activation

**Forward:** $y = \max(0, x)$

**Backward:** 
$$
\frac{\partial L}{\partial x} = \frac{\partial L}{\partial y} \odot \mathbb{1}_{x > 0}
$$

where $\odot$ is element-wise multiplication.

**Note on x=0:** Technically ReLU is not differentiable at 0. In practice, we set the gradient to 0 (or 1) at x=0.

In [ ]:
class ReLU(Layer):
    """ReLU activation: y = max(0, x)"""
    
    def forward(self, x):
        self.cache['x'] = x
        return np.maximum(0, x)
    
    def backward(self, grad_output):
        x = self.cache['x']
        # Gradient is 1 where x > 0, else 0
        grad_input = grad_output * (x > 0)
        return grad_input

### 2.3 Sigmoid Activation

**Forward:** $y = \sigma(x) = \frac{1}{1 + e^{-x}}$

**Backward:** 
$$
\frac{\partial \sigma}{\partial x} = \sigma(x)(1 - \sigma(x)) = y(1-y)
$$
$$
\frac{\partial L}{\partial x} = \frac{\partial L}{\partial y} \odot y(1-y)
$$

In [ ]:
class Sigmoid(Layer):
    """Sigmoid activation: y = 1 / (1 + exp(-x))"""
    
    def forward(self, x):
        y = 1 / (1 + np.exp(-np.clip(x, -500, 500)))  # Clip for numerical stability
        self.cache['y'] = y
        return y
    
    def backward(self, grad_output):
        y = self.cache['y']
        # dy/dx = y(1-y)
        grad_input = grad_output * y * (1 - y)
        return grad_input

### 2.4 Softmax with Cross-Entropy Loss

**Why combine them?** Numerical stability. Computing softmax then log then cross-entropy can cause overflow/underflow.

**Softmax:**
$$
\text{softmax}(x_i) = \frac{e^{x_i}}{\sum_j e^{x_j}}
$$

**Cross-Entropy Loss:**
$$
L = -\log(\text{softmax}(x_{\text{true class}}))
$$

**Log-Sum-Exp Trick** for numerical stability:
$$
\log\sum_i e^{x_i} = a + \log\sum_i e^{x_i - a}
$$
where $a = \max_i x_i$.

**Gradient:**
$$
\frac{\partial L}{\partial x_i} = \text{softmax}(x_i) - \mathbb{1}_{i = \text{true class}}
$$

This is a beautiful result: the gradient is just the difference between predicted probability and true label!

In [ ]:
class SoftmaxCrossEntropy(Layer):
    """Combined Softmax + Cross-Entropy for numerical stability.
    
    Forward: Computes loss
    Backward: Returns gradient w.r.t. logits
    """
    
    def forward(self, logits, labels):
        """
        Args:
            logits: (batch_size, num_classes)
            labels: (batch_size,) integer labels
        Returns:
            loss: scalar
        """
        batch_size = logits.shape[0]
        
        # Log-sum-exp trick for numerical stability
        logits_max = np.max(logits, axis=1, keepdims=True)
        logits_shifted = logits - logits_max
        exp_logits = np.exp(logits_shifted)
        sum_exp = np.sum(exp_logits, axis=1, keepdims=True)
        log_probs = logits_shifted - np.log(sum_exp)
        
        # Cross-entropy loss
        correct_log_probs = log_probs[np.arange(batch_size), labels]
        loss = -np.mean(correct_log_probs)
        
        # Cache for backward
        self.cache['probs'] = exp_logits / sum_exp
        self.cache['labels'] = labels
        self.cache['batch_size'] = batch_size
        
        return loss
    
    def backward(self):
        """
        Returns:
            grad_logits: (batch_size, num_classes)
        """
        probs = self.cache['probs']
        labels = self.cache['labels']
        batch_size = self.cache['batch_size']
        
        # Gradient: softmax(x) - one_hot(y)
        grad_logits = probs.copy()
        grad_logits[np.arange(batch_size), labels] -= 1
        grad_logits /= batch_size
        
        return grad_logits

## Part 3: Sequential Model Container

We need a container to stack layers sequentially, similar to `nn.Sequential` in PyTorch.

In [ ]:
class Sequential:
    """Sequential container for layers."""
    
    def __init__(self, *layers):
        self.layers = list(layers)
    
    def forward(self, x):
        """Forward pass through all layers."""
        for layer in self.layers:
            x = layer.forward(x)
        return x
    
    def backward(self, grad_output):
        """Backward pass through all layers in reverse."""
        for layer in reversed(self.layers):
            grad_output = layer.backward(grad_output)
        return grad_output
    
    def get_params_and_grads(self):
        """Return all parameters and gradients from all layers."""
        params = []
        grads = []
        for layer in self.layers:
            params.append(layer.get_params())
            grads.append(layer.get_grads())
        return params, grads

## Part 4: Optimizer

Implement SGD with momentum (from Day 4 theory).

**Update rule:**
$$
v_t = \beta v_{t-1} + (1-\beta)\nabla L(\theta_{t-1})
$$
$$
\theta_t = \theta_{t-1} - \alpha v_t
$$

In [ ]:
class SGD:
    """SGD optimizer with momentum.
    
    Args:
        lr: Learning rate
        momentum: Momentum coefficient (0 = no momentum)
    """
    
    def __init__(self, lr=0.01, momentum=0.9):
        self.lr = lr
        self.momentum = momentum
        self.velocities = None
    
    def step(self, params, grads):
        """Update parameters.
        
        Args:
            params: List of parameter dictionaries
            grads: List of gradient dictionaries
        """
        # Initialize velocities on first call
        if self.velocities is None:
            self.velocities = []
            for param_dict in params:
                velocity_dict = {}
                for key, value in param_dict.items():
                    velocity_dict[key] = np.zeros_like(value)
                self.velocities.append(velocity_dict)
        
        # Update parameters
        for param_dict, grad_dict, velocity_dict in zip(params, grads, self.velocities):
            for key in param_dict.keys():
                # v = beta * v + (1-beta) * grad
                velocity_dict[key] = self.momentum * velocity_dict[key] + (1 - self.momentum) * grad_dict[key]
                # theta = theta - lr * v
                param_dict[key] -= self.lr * velocity_dict[key]

## Part 5: Numerical Gradient Checking

Before training, we **must** verify our analytical gradients are correct.

**Finite Difference Approximation:**
$$
\frac{\partial L}{\partial \theta} \approx \frac{L(\theta + \epsilon) - L(\theta - \epsilon)}{2\epsilon}
$$

This is O(ε²) accurate (two-sided difference).

**Relative Error:**
$$
\text{error} = \frac{||\nabla_{\text{analytical}} - \nabla_{\text{numerical}}||_2}{||\nabla_{\text{analytical}}||_2 + ||\nabla_{\text{numerical}}||_2}
$$

**Good threshold:** error < 1e-7

In [ ]:
def numerical_gradient_check(layer, x, grad_output, epsilon=1e-5):
    """
    Check analytical gradients against numerical gradients.
    
    Args:
        layer: Layer to check
        x: Input data
        grad_output: Gradient from next layer
        epsilon: Finite difference step size
    """
    # Forward and backward pass
    output = layer.forward(x)
    analytical_grad = layer.backward(grad_output)
    
    # Numerical gradient w.r.t. input
    numerical_grad = np.zeros_like(x)
    
    # Compute numerical gradient element by element
    it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])
    count = 0
    while not it.finished and count < 10:  # Only check first 10 elements for speed
        ix = it.multi_index
        
        old_value = x[ix]
        
        # f(x + epsilon)
        x[ix] = old_value + epsilon
        out_plus = layer.forward(x)
        loss_plus = np.sum(out_plus * grad_output)
        
        # f(x - epsilon)
        x[ix] = old_value - epsilon
        out_minus = layer.forward(x)
        loss_minus = np.sum(out_minus * grad_output)
        
        # Numerical gradient
        numerical_grad[ix] = (loss_plus - loss_minus) / (2 * epsilon)
        
        # Restore
        x[ix] = old_value
        it.iternext()
        count += 1
    
    # Compute relative error
    numerator = np.linalg.norm(analytical_grad - numerical_grad)
    denominator = np.linalg.norm(analytical_grad) + np.linalg.norm(numerical_grad)
    relative_error = numerator / (denominator + 1e-8)
    
    print(f"Relative error: {relative_error:.2e}")
    if relative_error < 1e-7:
        print("✓ Gradient check PASSED")
    elif relative_error < 1e-5:
        print("⚠ Gradient check ACCEPTABLE (might be OK for some layers)")
    else:
        print("✗ Gradient check FAILED")
    
    return relative_error

### Test Gradient Checking on Linear Layer

In [ ]:
print("Testing Linear layer gradient...")
linear = Linear(5, 3)
x_test = np.random.randn(2, 5)
grad_out_test = np.random.randn(2, 3)
numerical_gradient_check(linear, x_test, grad_out_test)

print("\nTesting ReLU gradient...")
relu = ReLU()
x_test = np.random.randn(2, 5)
grad_out_test = np.random.randn(2, 5)
numerical_gradient_check(relu, x_test, grad_out_test)

print("\nTesting Sigmoid gradient...")
sigmoid = Sigmoid()
x_test = np.random.randn(2, 5)
grad_out_test = np.random.randn(2, 5)
numerical_gradient_check(sigmoid, x_test, grad_out_test)

## Part 6: Load MNIST Dataset

In [ ]:
# Load MNIST
print("Loading MNIST dataset...")
mnist = fetch_openml('mnist_784', version=1, parser='auto')
X, y = mnist['data'], mnist['target'].astype(int)

# Convert to numpy arrays
X = np.array(X, dtype=np.float32) / 255.0  # Normalize to [0, 1]
y = np.array(y, dtype=np.int64)

# Split into train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=10000, random_state=42)

print(f"Training set: {X_train.shape}")
print(f"Test set: {X_test.shape}")

# Visualize a few examples
fig, axes = plt.subplots(1, 5, figsize=(12, 3))
for i, ax in enumerate(axes):
    ax.imshow(X_train[i].reshape(28, 28), cmap='gray')
    ax.set_title(f"Label: {y_train[i]}")
    ax.axis('off')
plt.tight_layout()
plt.show()

## Part 7: Build and Train Neural Network

We'll build a simple 3-layer MLP:
- Input: 784 (28×28 pixels)
- Hidden 1: 128 neurons with ReLU
- Hidden 2: 64 neurons with ReLU
- Output: 10 classes

**Goal:** Achieve >90% test accuracy

In [ ]:
# Build model
model = Sequential(
    Linear(784, 128),
    ReLU(),
    Linear(128, 64),
    ReLU(),
    Linear(64, 10)
)

# Loss function
loss_fn = SoftmaxCrossEntropy()

# Optimizer
optimizer = SGD(lr=0.1, momentum=0.9)

print("Model architecture:")
print("784 -> [Linear] -> 128 -> [ReLU] -> 128 -> [Linear] -> 64 -> [ReLU] -> 64 -> [Linear] -> 10")

### Training Loop

In [ ]:
def train_epoch(model, X, y, loss_fn, optimizer, batch_size=128):
    """Train for one epoch."""
    num_samples = X.shape[0]
    num_batches = num_samples // batch_size
    
    total_loss = 0
    correct = 0
    
    # Shuffle data
    indices = np.random.permutation(num_samples)
    X_shuffled = X[indices]
    y_shuffled = y[indices]
    
    for i in range(num_batches):
        # Get batch
        start_idx = i * batch_size
        end_idx = start_idx + batch_size
        X_batch = X_shuffled[start_idx:end_idx]
        y_batch = y_shuffled[start_idx:end_idx]
        
        # Forward pass
        logits = model.forward(X_batch)
        loss = loss_fn.forward(logits, y_batch)
        
        # Backward pass
        grad_logits = loss_fn.backward()
        model.backward(grad_logits)
        
        # Update parameters
        params, grads = model.get_params_and_grads()
        optimizer.step(params, grads)
        
        # Track metrics
        total_loss += loss
        preds = np.argmax(logits, axis=1)
        correct += np.sum(preds == y_batch)
    
    avg_loss = total_loss / num_batches
    accuracy = correct / (num_batches * batch_size)
    
    return avg_loss, accuracy


def evaluate(model, X, y, loss_fn, batch_size=128):
    """Evaluate on test set."""
    num_samples = X.shape[0]
    num_batches = num_samples // batch_size
    
    total_loss = 0
    all_preds = []
    
    for i in range(num_batches):
        start_idx = i * batch_size
        end_idx = start_idx + batch_size
        X_batch = X[start_idx:end_idx]
        y_batch = y[start_idx:end_idx]
        
        logits = model.forward(X_batch)
        loss = loss_fn.forward(logits, y_batch)
        
        total_loss += loss
        preds = np.argmax(logits, axis=1)
        all_preds.extend(preds)
    
    avg_loss = total_loss / num_batches
    accuracy = accuracy_score(y[:len(all_preds)], all_preds)
    
    return avg_loss, accuracy, np.array(all_preds)

In [ ]:
# Training
num_epochs = 10
train_losses = []
train_accs = []
test_losses = []
test_accs = []

print("Starting training...\n")
start_time = time.time()

for epoch in range(num_epochs):
    # Train
    train_loss, train_acc = train_epoch(model, X_train, y_train, loss_fn, optimizer, batch_size=128)
    train_losses.append(train_loss)
    train_accs.append(train_acc)
    
    # Evaluate
    test_loss, test_acc, _ = evaluate(model, X_test, y_test, loss_fn, batch_size=128)
    test_losses.append(test_loss)
    test_accs.append(test_acc)
    
    print(f"Epoch {epoch+1}/{num_epochs} - Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f} | "
          f"Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.4f}")

end_time = time.time()
print(f"\nTraining completed in {end_time - start_time:.2f} seconds")
print(f"Final test accuracy: {test_accs[-1]:.4f}")

### Visualize Training Progress

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Loss plot
ax1.plot(train_losses, label='Train Loss', marker='o')
ax1.plot(test_losses, label='Test Loss', marker='s')
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Loss')
ax1.set_title('Training and Test Loss (NumPy Implementation)')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Accuracy plot
ax2.plot(train_accs, label='Train Accuracy', marker='o')
ax2.plot(test_accs, label='Test Accuracy', marker='s')
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Accuracy')
ax2.set_title('Training and Test Accuracy (NumPy Implementation)')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### Confusion Matrix

In [ ]:
# Get predictions on full test set
_, _, test_preds = evaluate(model, X_test, y_test, loss_fn, batch_size=128)
y_test_subset = y_test[:len(test_preds)]

# Confusion matrix
cm = confusion_matrix(y_test_subset, test_preds)

plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix (NumPy Implementation)')
plt.show()

# Per-class accuracy
print("\nPer-class accuracy:")
for i in range(10):
    class_acc = cm[i, i] / np.sum(cm[i, :])
    print(f"Digit {i}: {class_acc:.4f}")

## Part 8: Comparison with PyTorch

Let's compare our NumPy implementation with PyTorch in terms of:
1. Performance (accuracy)
2. Training speed
3. Code complexity

In [ ]:
# Convert data to PyTorch tensors
X_train_torch = torch.FloatTensor(X_train)
y_train_torch = torch.LongTensor(y_train)
X_test_torch = torch.FloatTensor(X_test)
y_test_torch = torch.LongTensor(y_test)

# Create DataLoaders
train_dataset = TensorDataset(X_train_torch, y_train_torch)
test_dataset = TensorDataset(X_test_torch, y_test_torch)
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=128)

# Define PyTorch model
class PyTorchMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(784, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 10)
        self.relu = nn.ReLU()
    
    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Initialize model
pytorch_model = PyTorchMLP()
criterion = nn.CrossEntropyLoss()
optimizer_torch = optim.SGD(pytorch_model.parameters(), lr=0.1, momentum=0.9)

print("PyTorch model architecture:")
print(pytorch_model)

In [ ]:
# Train PyTorch model
print("\nTraining PyTorch model...\n")
start_time = time.time()

pytorch_train_losses = []
pytorch_test_losses = []
pytorch_train_accs = []
pytorch_test_accs = []

for epoch in range(num_epochs):
    # Training
    pytorch_model.train()
    train_loss = 0
    correct = 0
    total = 0
    
    for X_batch, y_batch in train_loader:
        optimizer_torch.zero_grad()
        outputs = pytorch_model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer_torch.step()
        
        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += y_batch.size(0)
        correct += predicted.eq(y_batch).sum().item()
    
    pytorch_train_losses.append(train_loss / len(train_loader))
    pytorch_train_accs.append(correct / total)
    
    # Testing
    pytorch_model.eval()
    test_loss = 0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            outputs = pytorch_model(X_batch)
            loss = criterion(outputs, y_batch)
            
            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += y_batch.size(0)
            correct += predicted.eq(y_batch).sum().item()
    
    pytorch_test_losses.append(test_loss / len(test_loader))
    pytorch_test_accs.append(correct / total)
    
    print(f"Epoch {epoch+1}/{num_epochs} - Train Loss: {pytorch_train_losses[-1]:.4f}, Train Acc: {pytorch_train_accs[-1]:.4f} | "
          f"Test Loss: {pytorch_test_losses[-1]:.4f}, Test Acc: {pytorch_test_accs[-1]:.4f}")

pytorch_time = time.time() - start_time
print(f"\nPyTorch training completed in {pytorch_time:.2f} seconds")
print(f"Final test accuracy: {pytorch_test_accs[-1]:.4f}")

### Side-by-Side Comparison

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Loss comparison
axes[0, 0].plot(train_losses, label='NumPy Train', marker='o', linestyle='--')
axes[0, 0].plot(pytorch_train_losses, label='PyTorch Train', marker='s')
axes[0, 0].set_xlabel('Epoch')
axes[0, 0].set_ylabel('Loss')
axes[0, 0].set_title('Training Loss Comparison')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

axes[0, 1].plot(test_losses, label='NumPy Test', marker='o', linestyle='--')
axes[0, 1].plot(pytorch_test_losses, label='PyTorch Test', marker='s')
axes[0, 1].set_xlabel('Epoch')
axes[0, 1].set_ylabel('Loss')
axes[0, 1].set_title('Test Loss Comparison')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# Accuracy comparison
axes[1, 0].plot(train_accs, label='NumPy Train', marker='o', linestyle='--')
axes[1, 0].plot(pytorch_train_accs, label='PyTorch Train', marker='s')
axes[1, 0].set_xlabel('Epoch')
axes[1, 0].set_ylabel('Accuracy')
axes[1, 0].set_title('Training Accuracy Comparison')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

axes[1, 1].plot(test_accs, label='NumPy Test', marker='o', linestyle='--')
axes[1, 1].plot(pytorch_test_accs, label='PyTorch Test', marker='s')
axes[1, 1].set_xlabel('Epoch')
axes[1, 1].set_ylabel('Accuracy')
axes[1, 1].set_title('Test Accuracy Comparison')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Summary table
print("\n" + "="*60)
print("COMPARISON SUMMARY")
print("="*60)
print(f"{'Metric':<30} {'NumPy':>15} {'PyTorch':>15}")
print("-"*60)
print(f"{'Final Test Accuracy':<30} {test_accs[-1]:>15.4f} {pytorch_test_accs[-1]:>15.4f}")
print(f"{'Final Test Loss':<30} {test_losses[-1]:>15.4f} {pytorch_test_losses[-1]:>15.4f}")
print(f"{'Training Time (seconds)':<30} {end_time - start_time:>15.2f} {pytorch_time:>15.2f}")
print("="*60)
print(f"\nSpeed ratio (NumPy/PyTorch): {(end_time - start_time) / pytorch_time:.2f}x")
print(f"Accuracy difference: {abs(test_accs[-1] - pytorch_test_accs[-1]):.4f}")

## Part 9: Key Takeaways and Reflections

### What We Learned

1. **Automatic Differentiation:**
   - Reverse-mode autodiff computes gradients in O(n) time by propagating vector-Jacobian products backward
   - We never need to explicitly compute full Jacobian matrices
   - Each layer caches its inputs during forward pass for use in backward pass

2. **Layer Implementation:**
   - Linear layer: Simple matrix multiplication with careful handling of batched inputs
   - Activation functions: Element-wise operations with straightforward gradients
   - Softmax + Cross-Entropy: Combined for numerical stability using log-sum-exp trick

3. **Numerical Gradient Checking:**
   - Essential for validating analytical gradients
   - Finite differences provide O(ε²) accurate approximation
   - Relative error < 1e-7 indicates correct implementation

4. **Performance:**
   - NumPy implementation is slower than PyTorch (no GPU acceleration, less optimized)
   - But achieves comparable accuracy, validating our implementation
   - PyTorch provides highly optimized BLAS operations and GPU support

### Why This Matters

Understanding neural networks at this level allows you to:
- Debug training issues (gradient vanishing/explosion, numerical instability)
- Implement custom layers and operations
- Understand what PyTorch/TensorFlow do under the hood
- Optimize performance by knowing computational bottlenecks
- Design novel architectures with confidence

### Next Steps

**Optional Extensions:**
1. Implement Batch Normalization layer
2. Add Dropout layer
3. Implement Convolutional layer using im2col
4. Add Adam optimizer
5. Implement learning rate scheduling
6. Profile code to find bottlenecks

**Moving Forward:**
- Day 6: Transfer Learning with pretrained models
- Day 7: Comprehensive model evaluation
- You now have the foundation to understand any neural network architecture!

## Part 10: (Optional) Advanced Implementation - Batch Normalization

If you have extra time, try implementing Batch Normalization from scratch.

**Forward:**
$$
\hat{x} = \frac{x - \mu_B}{\sqrt{\sigma_B^2 + \epsilon}}
$$
$$
y = \gamma \hat{x} + \beta
$$

**Backward:** (Left as an exercise - derive the gradients!)

In [ ]:
class BatchNorm(Layer):
    """Batch Normalization layer (1D).
    
    Args:
        num_features: Number of features (channels)
        eps: Small constant for numerical stability
        momentum: Momentum for running mean/var
    """
    
    def __init__(self, num_features, eps=1e-5, momentum=0.1):
        super().__init__()
        self.eps = eps
        self.momentum = momentum
        
        # Learnable parameters
        self.params['gamma'] = np.ones(num_features)
        self.params['beta'] = np.zeros(num_features)
        
        # Running statistics (for inference)
        self.running_mean = np.zeros(num_features)
        self.running_var = np.ones(num_features)
        
        # Initialize gradients
        self.grads['gamma'] = np.zeros_like(self.params['gamma'])
        self.grads['beta'] = np.zeros_like(self.params['beta'])
        
        self.training = True
    
    def forward(self, x):
        """
        Args:
            x: (batch_size, num_features)
        Returns:
            y: (batch_size, num_features)
        """
        # TODO: Implement forward pass
        # Hint: Compute batch statistics, normalize, scale and shift
        raise NotImplementedError("Exercise: Implement BatchNorm forward pass")
    
    def backward(self, grad_output):
        """
        Args:
            grad_output: dL/dy, shape (batch_size, num_features)
        Returns:
            grad_input: dL/dx, shape (batch_size, num_features)
        """
        # TODO: Implement backward pass
        # Hint: This is challenging! Derive gradients w.r.t. x, gamma, beta
        raise NotImplementedError("Exercise: Implement BatchNorm backward pass")

# TODO: Test BatchNorm implementation
# TODO: Add BatchNorm to the model and compare performance

---

## Summary

Congratulations! You've implemented a complete neural network library from scratch using only NumPy. You now understand:

✅ Computational graphs and reverse-mode autodiff  
✅ Forward and backward propagation  
✅ Layer implementations (Linear, ReLU, Sigmoid, Softmax)  
✅ Numerical gradient checking  
✅ Training loops and optimization  
✅ How PyTorch works under the hood  

**Achievement:** >90% accuracy on MNIST with your own implementation!

**Time spent:** ~4-5 hours (if you completed all sections)

**Next:** Day 6 - Transfer Learning with Pretrained Models